In [1]:
#栅格计算
from osgeo import gdal
import numpy as np
import glob
import os
import re

In [2]:
def read_img(filename):
        dataset = gdal.Open(filename)
        im_width = dataset.RasterXSize #栅格矩阵的列数
        im_height = dataset.RasterYSize #栅格矩阵的行数
        im_geotrans = dataset.GetGeoTransform() #仿射矩阵
        im_proj = dataset.GetProjection() #地图投影
        im_data = dataset.ReadAsArray(0,0,im_width,im_height)
        del dataset
        return im_proj,im_geotrans,im_data

def write_img(filename,im_proj,im_geotrans,im_data):
        if "int8" in im_data.dtype.name:
            datatype = gdal.GDT_Byte
        if "int16" in im_data.dtype.name:
            datatype = gdal.GDT_UInt16
        else:
            datatype = gdal.GDT_Float32

        if len(im_data.shape) == 3:
            im_bands, im_height, im_width = im_data.shape
        else:
            im_bands, (im_height, im_width) = 1, im_data.shape

        driver = gdal.GetDriverByName("GTiff")  # 数据类型必须有，因为要计算需要多大内存空间
        dataset = driver.Create(filename, im_width, im_height, im_bands, datatype)

        dataset.SetGeoTransform(im_geotrans)  # 写入仿射变换参数
        dataset.SetProjection(im_proj)  # 写入投影

        if im_bands == 1:
            dataset.GetRasterBand(1).WriteArray(im_data)  # 写入数组数据
        else:
            for i in range(im_bands):
                dataset.GetRasterBand(i + 1).WriteArray(im_data[i])
        del dataset

In [45]:
aod_path  = 'F:\\SCI\\CESM\\'
aod_list = glob.glob(os.path.join(aod_path, 'CESM2.HIS.AODVIS.*.tif'))
proj = read_img("F:\\SCI\\CESM\\CESM2.HIS.AODVIS.1980-01.tif")[0]
geotrans = read_img("F:\\SCI\\CESM\\CESM2.HIS.AODVIS.1980-01.tif")[1]
annualmean = []


In [46]:
#输入三个文件路径，返回计算完成后的文件
def cal(aod, aodt, mod, out):
    aod_arr  = read_img(aod)[2]
    aodt_arr = read_img(aodt)[2]
    mod_arr  = read_img(mod)[2]/1000
    #将三个文件中的空值转换为nan
    aod_arr[np.where(aod_arr<=-3.3e+38)] = np.nan
    aodt_arr[np.where(aodt_arr<=-3.3e+38)] = np.nan
    mod_arr[np.where(mod_arr == 0)] = np.nan

    pm25 = aodt_arr*mod_arr/aod_arr

    write_img(out, proj, geotrans, pm25)

In [47]:
cal('F:\\SCI\\CESM\\CESM2.HIS.AODVIS.1980-01.tif', 'F:\\SCI\\CESM\\CESM2.hist.AODT.1980-01.tif' ,'C:\\Users\\DJL\\Desktop\\dataanalysis\\modresrs\\200301modsrs.tif', 'F:\\test\\pm25test.tif')

In [3]:
t1 = read_img('F:\\test\\pm25test.tif')[2]
print(np.nanmean(t1))
t1


0.1773009


array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [51]:
for y_idx in range(2003, 2015):
    for m_idx in range(1, 13):
        for aod_name in aod_list:
            # 获取文件名中日期信息
            #name = re.findall(r'MOD16A2GF.A(\d{7}).h28v06', path.basename(et_name))[0]
            #date = datetime.datetime.strptime(name, '%Y%j')
            #year, month, day = date.year, date.month, date.day
            date = re.findall(r'....-..',aod_name)
            syear = date[0][0:4]
            smonth = date[0][-2:]
            year = int(date[0][0:4])
            month = int(date[0][-2:])
            if year == y_idx:
                if month == m_idx:
                    # a1 = read_img(f"F:\\SCI\\MOD\\DBAOD550\\tif\\MYD.DB.AOD550.{syear}{smonth}.tif")[2]
                    # a1 = a1.astype(np.float32)
                    # a1[np.where(a1 <= -3.3e+38)]=np.nan
                    # # 同一年相同月份的影像存放一起并插入各景影像数据在各月份中所占权重
                    # # 根据所占月份时间长短确定权重（暂时设为1）
                    # if m_idx == 1:
                    #     m_array = a1
                    # elif m_idx ==2:
                    #     m_array = np.stack([m_array,a1],axis=2)
                    # else:
                    #     m_array = np.insert(m_array,0,a1,axis=2)
                    # break
                    mod_path = f"C:\\Users\\DJL\\Desktop\\dataanalysis\\modresrs\\{syear}{smonth}modsrs.tif"
                    aod_path = f"F:\\SCI\\CESM\\CESM2.HIS.AODVIS.{syear}-{smonth}.tif"
                    aodt_path = f"F:\\SCI\\CESM\\CESM2.hist.AODT.{syear}-{smonth}.tif"
                    out_path = f"C:\\Users\\DJL\\Desktop\\dataanalysis\\pm25\\{syear}-{smonth}pm25.tif"
                    cal(aod_path, aodt_path, mod_path, out_path)
                    break
